A faire : 
- parcourir toutes les dates
- cliquer sur un jour 
- cliquer sur un tournoi
- aller dans l'onglet "grille" (s'il y en a) et lire les informations : 
- mettre dans un tableau, pour chaque joueur le nom du tournoi, le classement du joueur dans le tournoi, contre qui il a joué (avec le score) 

sous la forme d'un dictionnaire {'nom du joueur' : untel, 'elo': 2000, 'tournois' : [{'nom' : Tournoi du pin, 'date' : 15/04/2022, 'classement' : 13}, {'nom' : Tournoi du chêne, 'date' : 15/02/2021, 'classement' : 8}] , 'adversaires' : [{'nom' : untel, 'elo' : 1900, 'score' : perdu, 'couleur' : noirs, 'date' : 15/09/2020}, {'nom' : untel, 'elo' : 1500, 'score' : perdu, 'couleur' : noirs, 'tournoi' : Nationale 3, 'date' : 13/08/2021}]}

#indication : =/- : gagné, nul, perdu , 4 : chiffre de l’adversaire, B/N : blancs ou noirs 

In [1]:
import requests

#out = requests.get("http://echecs.asso.fr/FicheTournoi.aspx?Ref=55579")
#out = requests.get("http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/55579/55579&Action=Ga")
out = requests.get("http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/55605/55605&Action=Berger")
out.headers
print(out.content.decode())



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" >
<head id="ctl00_MasterHeader"><meta http-equiv="content-type" content="application/xhtml+xml; charset=utf-8" /><meta name="keywords" /><meta name="description" /><title>
	F&eacute;d&eacute;ration Fran&ccedil;aise des &Eacute;checs
</title><link type="text/css" rel="stylesheet" href="http://fonts.googleapis.com/css?family=Source+Sans+Pro:200,300,400,600" /><link href="http://fonts.googleapis.com/css?family=Oswald:300,400,700" rel="stylesheet" type="text/css" /><link type="text/css" rel="stylesheet" href="ffe.css?version=6" /><link type="text/css" rel="stylesheet" href="old.css?version=1" /><link rel="shortcut icon" type="image/x-icon" href="images/ffe16.ico" />
  <script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o)

In [1]:
#RECUPERATION DES DONNES POUR UNE GRILLE BERGER

import requests
from bs4 import BeautifulSoup

def recup_berger(number):#number=numéro d'une grille berger
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Berger"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    #récupération des joueurs
    j=[]
    for x in soup.find_all('td'):
        if x.has_attr("class") and x['class'][0] == "papi_border_l" :
            j.append(x.string.lower())
    joueur=j[1:]
    
    #récupération des elo
    e=[]
    i=0
    for x in soup.find_all('tr'):
        if x.has_attr("class") and x['class'][0] == "papi_liste_c":
            for k,y in enumerate(x.find_all('td')):
                if (k-3)%15==0 and i<len(joueur)+1:
                    e.append(y.string)
                    i+=1
    elo=dict()
    for i in range(len(joueur)):
        elo[joueur[i]]=e[1:][i]
    
    #récupération des scores relatifs sous forme de dictionnaire
    dic=dict()
    for i in range(len(joueur)):
        p=[]
        for x in soup.find_all('tr'):
            if x.has_attr("class") and x['class'][0] == "papi_liste_c":
                for k,y in enumerate(x.find_all('td')):
                    if (k-4-i)%15==0:
                        p.append(y.string)
        perf_colonne=p[1:len(joueur)+1]
        dic[joueur[i]]=perf_colonne
        #avant on a récupéré tous les modulos 15, ie on a les scores en colonne, on doit les présenter en ligne   
    dictionnaire=dict()
    for i in range(8):
        dico_joueur=dictionnaire[joueur[i]]=dict()
        for j in range(8):
            if j!=i:
                dico_joueur[joueur[j]]=dic[joueur[j]][i]
    return(joueur,elo,dictionnaire)       
     

       

In [51]:
import requests
from bs4 import BeautifulSoup

link = "http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/30384/30384&Action=Ga"
soup = BeautifulSoup(requests.get(link).text, 'html.parser')

print(soup)

for x in soup.find_all('div'):
    if x.has_attr("class") and x['class'][0] == "papi_joueur_box":
        print(x.b.string)
        
adversaires = []
elo = []
perf = []
couleur = []
résultat = []

for x in soup.find_all('div'):
    if x.has_attr("class") and x.has_attr("align") and x['align'] == "center":
        joueur = []
        for k,y in enumerate(x.find_all('td')):
            print(y.string)
            if (k-16)%13==0:
                print(f"c'est {k}")
                joueur.append(y.string)
            if (k-17)%13==0:
                z=str(y.string).split("\xa0")
                elo.append(z)
            if (k-23)%13==0:
                perf.append(y.string)
            if (k-12)%13==0:
                couleur.append(y.string)
            if k%13==0:
                résultat.append(y.string)
        adversaires.append(joueur) 

print(adversaires)
print(couleur)
print(résultat)
print(elo)
print(perf)
       

dico_adversaires = dict()
for x in adversaires :
    dico_adversaires[x[0]]=[x[1]]
    for k in range(2, len(x)):
        dico_adversaires[x[0]].append(x[k])
        
print(dico_adversaires)
print(elo)
                


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="ctl00_MasterHeader"><meta content="application/xhtml+xml; charset=utf-8" http-equiv="content-type"/><meta name="keywords"/><meta name="description"/><title>
	Fédération Française des Échecs
</title><link href="http://fonts.googleapis.com/css?family=Source+Sans+Pro:200,300,400,600" rel="stylesheet" type="text/css"/><link href="http://fonts.googleapis.com/css?family=Oswald:300,400,700" rel="stylesheet" type="text/css"/><link href="ffe.css?version=6" rel="stylesheet" type="text/css"/><link href="old.css?version=1" rel="stylesheet" type="text/css"/><link href="images/ffe16.ico" rel="shortcut icon" type="image/x-icon"/>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.a

In [53]:
#savoir lire ce genre de grille américaine
link = "http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/30384/30384&Action=Ga"
soup = BeautifulSoup(requests.get(link).text, 'html.parser')

print(soup)



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="ctl00_MasterHeader"><meta content="application/xhtml+xml; charset=utf-8" http-equiv="content-type"/><meta name="keywords"/><meta name="description"/><title>
	Fédération Française des Échecs
</title><link href="http://fonts.googleapis.com/css?family=Source+Sans+Pro:200,300,400,600" rel="stylesheet" type="text/css"/><link href="http://fonts.googleapis.com/css?family=Oswald:300,400,700" rel="stylesheet" type="text/css"/><link href="ffe.css?version=6" rel="stylesheet" type="text/css"/><link href="old.css?version=1" rel="stylesheet" type="text/css"/><link href="images/ffe16.ico" rel="shortcut icon" type="image/x-icon"/>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.a

In [5]:
base = dict()

number = 55579

#checker si la page est américaine 
def american(number):
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    for x in soup.find_all('span'):
        if x.string == "Désolé, le fichier n'existe pas...":
            #print(x.string)
            return False
        else :
            print("c'est ok")
            return True 


#pour avoir la date, le nom, la cadence, et les rondes du tournoi
def infos_tournoi(number): #problème : le tournoi peut ne pas avoir de date 
    infos = dict()
    link_infos = f"http://echecs.asso.fr/FicheTournoi.aspx?Ref={number}"
    soup_infos = BeautifulSoup(requests.get(link_infos).text, 'html.parser')
    for x in soup_infos.find_all('span'):
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelDates":
            infos["date"] = x.string   
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNom":
            infos["nom"] = x.string  
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNbrRondes":
            infos["nb_rondes"] = x.string
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelCadence":
            infos["cadence"] = x.string 
    return infos



#dans la boucle, si la date c'est demain, on arrête la boucle 

from datetime import datetime

months = {"janvier":1, "février":2, "mars":3, "avril":4,
       "mai":5, "juin":6, "juillet":7, "août":8,
       "septembre":9, "octobre":10, "novembre":11, "décembre":12}

def stop_boucle(date): #fonction qui renvoie true si on doit arrêter la boucle
    liste_date = date.strip('-').split(' ')
    for k in range(len(liste_date)):
        if k%5==1:
            #print(liste_date[k])
            jour = liste_date[k]
        if k%5==2:
            #print(k)
            mois = months[liste_date[k]]
        if k%5==3:
            année = liste_date[k]
    jour_now, mois_now, année_now = datetime.today().day, datetime.today().month, datetime.today().year
    if int(année)>int(année_now):
        return True
    elif int(année)==int(année_now):
        if int(mois)>int(mois_now):
            return True
        elif int(mois)==int(mois_now):
            if int(jour)>int(jour_now):
                return True
    return False
        
print(stop_boucle('mercredi 23 juin 2022 - samedi 8 juillet 2022'))
    

adversaires = []
elo = []
perf = []

for x in soup.find_all('div'):
    if x.has_attr("class") and x.has_attr("align") and x['align'] == "center":
        joueur = []
        for k,y in enumerate(x.find_all('td')):
            if (k-16)%13==0:
                joueur.append(y.string)
            if (k-17)%13==0:
                elo.append(y.string)
            if (k-24)%13==0:
                perf.append(y.string)
            print(y.string)
        adversaires.append(joueur) 
        
print(base)


True


NameError: name 'soup' is not defined

In [5]:
#tentative de SQL

import sqlite3

CreateDataBase = sqlite3.connect('MyDataBase.db')

QueryCurs = CreateDataBase.cursor()

def CreateTable():
    QueryCurs.execute('''CREATE TABLE Joueurs
    (id INTEGER PRIMARY KEY, Nom TEXT,Elo TEXT, Nationalité TEXT)''')

def AddEntry(Nom,Elo,Nationalité):
    QueryCurs.execute('''INSERT INTO Joueurs (Nom,Elo,Nationalité)
    VALUES (?,?,?)''',(Nom,Elo,Nationalité))



AddEntry('Tom','105','FRance')


CreateDataBase.commit()

QueryCurs.execute('SELECT * FROM Joueurs')

for i in QueryCurs:
    print ("\n")
    for j in i:
        print (j)

QueryCurs.close()

OperationalError: no such table: Joueurs

In [7]:
#marche à suivre
#faire une boucle sur les numéros de match
#while la date est encore à jour
#tester si la page est américaine ou berger 
#collecter les infos du tournoi et les mettre dans la table tournoi
#récupérer la liste des joueurs
#pour chaque player, tester s'il est déjà dans la base 
#récupérer les matchs et les mettre dans la base de donnée 

In [6]:
import sqlite3

#création de la base de données 
con = sqlite3.connect(":memory:")
con.executescript("""
create table player (id integer not NULL primary key autoincrement, name text not NULL); 
create table tournois (id integer not NULL primary key autoincrement, name text not NULL, date text not NULL, rondes integer, cadence string);
create table match (id integer primary key autoincrement, tournoi integer, black_player integer, elo_black string, white_player integer, elo_white string, winner integer);""")

#con.execute("insert into player(name) values (?)", ("philippe marchal",))
#con.execute("insert into player(id, name) values (?, ?)", (0, "pas d'adversaire"))

#for row in con.execute("select name from player where name = 'philippea marchal'"):
    #print(row)


    
#print(list(con.execute("select name from player where name = 'philippe marchal'")))[0]

#winner : 0 pour les noirs, 3 pour les noirs au temps, 1 pour les blancs, 4 pour les blancs au temps, 2 pour ex aequo

In [ ]:
#Commandes SQL

buffer = ""

print("Enter your SQL command to execute")
print("Enter a blank line to exit")

while True:
    line = input()
    if line =="":
        break
    buffer += line
    if sqlite3.complete_statement(buffer):
        try:
            buffer = buffer.strip()
            con.execute(buffer)
            for row in con.execute(buffer):
                print(row)
            
            if buffer.lstrip().upper().startswitch("SELECT"):
                print(cur.fetchall())
        except sqlite.Error as e :
            print("An error occured:", e.args[0] )
        buffer = ""


Enter your SQL command to execute
Enter a blank line to exit
select * from player


In [13]:
def players_ga(number):
    res = []
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    for x in soup.find_all('div'):
        if x.has_attr("class") and x['class'][0] == "papi_joueur_box":
            res.append(x.b.string)
            
    return res
            
            
def match_ga(number):
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    res = []
    adversaires = []
    elos = []
    couleurs = []
    résultats = []

    for x in soup.find_all('div'):
        if x.has_attr("class") and x.has_attr("align") and x['align'] == "center":
            joueur = []
            elo = []
            couleur = []
            résultat = []
            for k,y in enumerate(x.find_all('td')):
                if (k-16)%13==0:
                    joueur.append(y.string)
                if (k-17)%13==0:
                    z=str(y.string).split("\xa0")
                    elo.append(z)
                if (k-12)%13==0:
                    couleur.append(y.string)
                if k%13==0:
                    résultat.append(y.string)
            adversaires.append(joueur) 
            elos.append(elo)
            couleurs.append(couleur)
            résultats.append(résultat)
    #print((adversaires, elos, couleurs, résultats))
    déjà_vus = set()
    for k in range (len(adversaires)):
        for i in range(1,len(adversaires[k])):
            match = []
            try:
                match.append(adversaires[k][0])
                déjà_vus.add(adversaires[k][0])
                match.append(adversaires[k][i])
                match.append(elos[k][0])
                match.append(elos[k][i])
                match.append(couleurs[k][i-1])
                match.append(résultats[k][i])
                if adversaires[k][i] not in déjà_vus:
                    res.append(match)
            except IndexError:
                pass
    return res
                      
players_ga(55579)
match_ga(55200)
                

[['DGEBUADZE Alexandre',
  'DELPORTE Simon',
  ['2464', 'F'],
  ['1540', 'F'],
  'B',
  '1'],
 ['DGEBUADZE Alexandre',
  'ONBELET Steve',
  ['2464', 'F'],
  ['1765', 'F'],
  'N',
  '1'],
 ['DGEBUADZE Alexandre',
  'DESSOMME Francois',
  ['2464', 'F'],
  ['1967', 'F'],
  'B',
  '1'],
 ['DGEBUADZE Alexandre',
  'LEMIRE Axel',
  ['2464', 'F'],
  ['1781', 'F'],
  'N',
  '1'],
 ['DGEBUADZE Alexandre',
  'PUCHER Sebastien',
  ['2464', 'F'],
  ['2281', 'F'],
  'B',
  '½'],
 ['DGEBUADZE Alexandre',
  'DE BLOCK Yordi',
  ['2464', 'F'],
  ['2102', 'F'],
  'N',
  '1'],
 ['PUCHER Sebastien',
  'LECOMTE Patrick',
  ['2281', 'F'],
  ['1521', 'F'],
  'N',
  '1'],
 ['PUCHER Sebastien', 'FAROU Fabrice', ['2281', 'F'], ['1763', 'F'], 'B', '1'],
 ['PUCHER Sebastien',
  'BAUMEL Laurent',
  ['2281', 'F'],
  ['1817', 'F'],
  'N',
  '1'],
 ['PUCHER Sebastien',
  'BOUDRY William',
  ['2281', 'F'],
  ['2128', 'F'],
  'B',
  '1'],
 ['PUCHER Sebastien',
  'THIRION Marcel',
  ['2281', 'F'],
  ['2072', 'F'],
  'B'

In [ ]:
#on insère les infos d'un match américain dans les tables

def insere_ga(number):
    #inserer le tournoi
    date_tournoi = infos_tournoi(number)["date"]
    nom_tournoi = infos_tournoi(number)["nom"]
    nb_rondes_tournoi = infos_tournoi(number)["nb_rondes"]
    cadence_tournoi = infos_tournoi(number)["cadence"]
    #test si le nom ou la date est None
    con.execute("insert into tournois(name, date, rondes, cadence) values (?,?,?,?)", (nom_tournoi, date_tournoi, nb_rondes_tournoi, cadence_tournoi))
    
    #inserer les joueurs du tournoi
    for player in players_ga(number):
        p = player.lower()
        recherche = list(con.execute("select name from player where name = ?", (p,)))
        if recherche == []:
            con.execute("insert into player(name) values (?)", (player.lower(),))
    
    #inserer les matchs du tournoi
    for element in match_ga(number):
        print(element)
        id_tournoi = list(con.execute("select id from tournois where name = ?", (nom_tournoi,)))[0][0]
        if element[1]!= 'EXEMPT' and element[1]!=None and element[0]!='EXEMPT' and element[0]!=None:
            p_1 = list(con.execute("select id from player where name = ?", (element[0].lower(),)))
            p_2 = list(con.execute("select id from player where name = ?", (element[1].lower(),)))
            if p_1!=[] and p_2!=[]:
                player_1 = list(con.execute("select id from player where name = ?", (element[0].lower(),)))[0][0]
                player_2 = list(con.execute("select id from player where name = ?", (element[1].lower(),)))[0][0]
                if len(element[2])==2 :
                    elo_1 = element[2][0]+element[2][1]
                else :
                    elo_1 = 0
                if len(element[3])==2:
                    elo_2 = element[3][0]+element[3][1]
                else :
                    elo_2 = 0
                if element[4] == 'N':
                    black_player = player_1
                    elo_black = elo_1
                    white_player = player_2
                    elo_white = elo_2
                    if element[5]=='1' :
                        winner = 0
                    elif element[5]=='½' or element[5]=="=":
                        winner = 2
                    elif element[5]=='0':
                        winner = 1 
                    elif element[5]=='>':
                        winner = 3
                    elif element[5]=='<':
                        winner = 4
                else :
                    white_player = player_1
                    elo_white = elo_1
                    black_player = player_2
                    elo_black = elo_2
                    if element[5]=='1' :
                        winner = 1
                    elif element[5]=='½' or element[5]=="=":
                        winner = 2
                    elif element[5]=='0':
                        winner = 0
                    elif element[5]=='>':
                        winner = 4
                    elif element[5]=='<':
                        winner = 3
                con.execute("insert into match(tournoi, black_player, elo_black, white_player, elo_white, winner) values (?,?,?,?,?,?)", (id_tournoi, black_player, elo_black, white_player, elo_white, winner))


In [55]:
for row in con.execute("select * from match"):
    print(row)
    
for row in con.execute("select * from player"):
    print(row)
    
for row in con.execute("select * from tournois"):
    print(row)

In [67]:
#cellule à exécuter pour vider la base de donnée
con.execute("delete from player");
con.execute("delete from sqlite_sequence where name='player'");
for row in con.execute("select * from player"):
    print(row)
    
con.execute("delete from match");
con.execute("delete from sqlite_sequence where name='match'");
for row in con.execute("select * from match"):
    print(row)
    
con.execute("delete from tournois");
con.execute("delete from sqlite_sequence where name='tournois'");
for row in con.execute("select * from tournois"):
    print(row)

In [68]:
number = 30000
while number<30500:
    number+=1
    print(number)
    if american(number):
        try:
            insere_ga(number)
        except IntegrityError: #vérifier que les dates sont non nulles
            pass
        
for row in con.execute("select * from match"):
    print(row)
    
for row in con.execute("select * from player"):
    print(row)
    
for row in con.execute("select * from tournois"):
    print(row)

c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
['ANDREANI Thomas', 'FERRACCI-CECCALDI Armilia', ['1260', 'N'], ['1140', 'N'], 'B', '1']
['ANDREANI Thomas', 'DEFENDINI Henri', ['1260', 'N'], ['1250', 'N'], 'N', '1']
['ANDREANI Thomas', 'CARBUCCIA Chjara', ['1260', 'N'], ['1240', 'N'], 'B', '½']
['ANDREANI Thomas', 'EXEMPT', ['1260', 'N'], ['None'], 'B', '>']
['CARBUCCIA Chjara', 'PODVIN Antoine', ['1240', 'N'], ['1692', 'F'], 'N', '0']
['CARBUCCIA Chjara', 'CASANOVA Marcu', ['1240', 'N'], ['1180', 'N'], 'B', '1']
['CARBUCCIA Chjara', 'DEFENDINI Henri', ['1240', 'N'], ['1250', 'N'], 'N', '1']
['DEFENDINI Henri', 'CASANOVA Marcu', ['1250', 'N'], ['1180', 'N'], 'N', '1']
['DEFENDINI Henri', 'FERRACCI-CECCALDI Armilia', ['1250', 'N'], ['1140', 'N'], 'N', '1']
['CASANOVA Marcu', 'VOUAKOUANITOU Nathan', ['1180', 'N'], ['1290', 'N'], 'B', '1']
['CASANOVA Marcu', 'FERRACCI-CECCALDI

c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
['CASSIAN Nita', 'BRISEPIERRE Richard', ['1499', 'N'], ['1399', 'N'], 'N', '1']
['CASSIAN Nita', 'QUARTARARO Maxime', ['1499', 'N'], ['1500', 'N'], 'B', '1']
['CASSIAN Nita', 'QUARTARARO Pierre', ['1499', 'N'], ['2010', 'N'], 'B', '½']
['CASSIAN Nita', 'QUARTARARO Remi', ['1499', 'N'], ['1410', 'N'], 'N', '1']
['CASSIAN Nita', 'LAFAURE Etienne', ['1499', 'N'], ['1440', 'N'], 'B', '1']
['CASSIAN Nita', 'MOULIN Noel', ['1499', 'N'], ['1480', 'N'], 'N', '1']
['QUARTARARO Pierre', 'LAFAURE Etienne', ['2010', 'N'], ['1440', 'N'], 'N', '1']
['QUARTARARO Pierre', 'DADOU-MOURAD Mylan', ['2010', 'N'], ['1499', 'N'], 'B', '1']
['QUARTARARO Pierre', 'MOULIN Noel', ['2010', 'N'], ['1480', 'N'], 'B', '1']
['QUARTARARO Pierre

c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
['SUMETS Andrey', 'DEGARDIN Sylvain', ['2633', 'F'], ['2107', 'F'], 'B', '1']
['SUMETS Andrey', 'SCHWING Hans-Elmar', ['2633', 'F'], ['2287', 'F'], 'N', '1']
['SUMETS Andrey', 'KARPATCHEV Aleksandr', ['2633', 'F'], ['2442', 'F'], 'B', '1']
['SUMETS Andrey', 'ENCHEV Ivaljo', ['2633', 'F'], ['2443', 'F'], 'N', '½']
['SUMETS Andrey', 'IVANOV Mikhail M', ['2633', 'F'], ['2366', 'F'], 'B', '1']
['SUMETS Andrey', 'LERCH Patrice', ['2633', 'F'], ['2358', 'F'], 'B', '1']
['SUMETS Andrey', 'GULIYEV Namig', ['2633', 'F'], ['2559', 'F'], 'N', '1']
['SUMETS Andrey', 'MOMCHIL Nikolov', ['2633', 'F'], ['2515', 'F'], 'B', '½']
['SUMETS Andrey', 'LALIC Bogdan', ['2633', 'F'], ['2485', 'F'], 'N', '½']
['KARPATCHEV Aleksandr', 'LABADIE Jean-Jacques', ['2442', 'F'], ['1974', 'F'], 'N', '1']
['KARPATCHEV Aleksandr', 'GERARD Nicolas', ['2442', 'F'], ['2262', 'F'], 'B', '1']
['KARPATCHEV Aleksandr', 'MOMCHIL N

c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
['RIVIERE Etienne', 'ATLAN Yoram', ['1199', 'E'], ['1400', 'N'], 'N', '1']
['RIVIERE Etienne', 'LAPIERRE Kian', ['1199', 'E'], ['1009', 'E'], 'B', '1']
['RIVIERE Etienne', 'TRITON Maxime', ['1199', 'E'], ['1199', 'E'], 'N', '1']
['RIVIERE Etienne', 'PAYET Manuel', ['1199', 'E'], ['1199', 'E'], 'B', '1']
['RIVIERE Etienne', 'RINGAYEN Francinel', ['1199', 'E'], ['1361', 'F'], 'B', '½']
['RIVIERE Etienne', 'CARBALLAR Alexandre', ['1199', 'E'], ['1050', 'N'], 'N', '1']
['TRITON Maxime', 'RINGAYEN Francinel', ['1199', 'E'], ['1361', 'F'], 'B', '1']
['TRITON Maxime', 'FONTAINE Loic', ['1199', 'E'], ['1200', 'N'], 'N', '1']
['TRITON Maxime', 'CARBALLAR Alexandre', ['1199', 'E'], ['1050', 'N'], 'N', '1']
['TRITON Maxime', 'CAMINADE Lionel', ['1199', 'E'], ['1499', 'E'], 'N', '1']
['TRITON Maxime', 'PAYET Man

c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
['KARPATCHEV Aleksandr', 'HELFER Remi', ['2462', 'F'], ['2020', 'F'], 'B', '1']
['KARPATCHEV Aleksandr', 'STAHL Clement', ['2462', 'F'], ['2155', 'F'], 'N', '1']
['KARPATCHEV Aleksandr', 'DOLUHANOVA Evgeniya', ['2462', 'F'], ['2320', 'F'], 'B', '1']
['KARPATCHEV Aleksandr', 'LERCH Patrice', ['2462', 'F'], ['2374', 'F'], 'N', '½']
['KARPATCHEV Aleksandr', 'BELLAHCENE Bilel', ['2462', 'F'], ['2357', 'F'], 'B', '1']
['KARPATCHEV Aleksandr', 'SHIRAZI Kamran', ['2462', 'F'], ['2350', 'F'], 'B', '1']
['KARPATCHEV Aleksandr', 'SHCHEKACHEV Andrei', ['2462', 'F'], ['2550', 'F'], 'N', '½']
['SHCHEKACHEV Andrei', 'VIGNALE Stephane', ['2550', 'F'], ['2021', 'F'], 'N', '1']
['SHCHEKACHEV Andrei', 'RUFF Maximilian', ['2550', 'F'], ['2170', 'F'], 'B', '1']
['SHCHEKACHEV Andrei', 'BRAEUNING Rudolf Wilhelm', ['2550', 'F'], ['2344', 'F'], 'N', '½']
['SHCHEKACHEV Andrei', 'HAMID Emran', ['2550', 'F

...

c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
c'est ok
(1, 18, 9, '1140N', 4, '1260N', 1)
(2, 18, 4, '1260N', 6, '1250N', 0)
(3, 18, 5, '1240N', 4, '1260N', 2)
(4, 18, 5, '1240N', 8, '1692F', 1)
(5, 18, 7, '1180N', 5, '1240N', 1)
(6, 18, 5, '1240N', 6, '1250N', 0)
(7, 18, 6, '1250N', 7, '1180N', 0)
(8, 18, 6, '1250N', 9, '1140N', 0)
(9, 18, 10, '1290N', 7, '1180N', 1)
(10, 18, 7, '1180N', 9, '1140N', 0)
(11, 28, 23, '1550N', 11, '2130N', 1)
(12, 28, 11, '2130N', 29, '1700N', 0)
(13, 28, 18, '1620N', 11, '2130N', 1)
(14, 28, 11, '2130N', 15, '2120N', 0)
(15, 28, 19, '1780N', 11, '2130N', 1)
(16, 28, 11, '2130N', 12, '2070N', 2)
(17, 28, 14, '1990N', 11, '2130N', 1)
(18, 28, 22, '1510N', 12, '2070N', 1)
(19, 28, 12, '2070N', 18, '1620N', 1)
(20, 28, 

(1946, 182, 611, '799N', 596, '799N', 1)
(1947, 182, 596, '799N', 615, '999N', 0)
(1948, 182, 617, '799N', 597, '799N', 1)
(1949, 182, 597, '799N', 604, '999N', 0)
(1950, 182, 611, '799N', 597, '799N', 1)
(1951, 182, 620, '799N', 598, '799N', 1)
(1952, 182, 598, '799N', 599, '799N', 1)
(1953, 182, 609, '799N', 598, '799N', 1)
(1954, 182, 613, '799N', 598, '799N', 1)
(1955, 182, 607, '799N', 599, '799N', 0)
(1956, 182, 599, '799N', 626, '799N', 0)
(1957, 182, 599, '799N', 608, '799N', 0)
(1958, 182, 609, '799N', 600, '1499N', 1)
(1959, 182, 600, '1499N', 602, '799N', 1)
(1960, 182, 606, '799N', 600, '1499N', 0)
(1961, 182, 600, '1499N', 618, '799N', 0)
(1962, 182, 608, '799N', 600, '1499N', 1)
(1963, 182, 602, '799N', 601, '799N', 0)
(1964, 182, 601, '799N', 609, '799N', 1)
(1965, 182, 601, '799N', 625, '799N', 0)
(1966, 182, 612, '799N', 601, '799N', 1)
(1967, 182, 606, '799N', 601, '799N', 1)
(1968, 182, 602, '799N', 603, '999N', 2)
(1969, 182, 624, '799N', 603, '999N', 1)
(1970, 182,

(3190, 224, 944, '1951F', 969, '1714F', 0)
(3191, 224, 944, '1951F', 954, '1760F', 2)
(3192, 224, 976, '1840F', 944, '1951F', 1)
(3193, 224, 958, '1775F', 944, '1951F', 1)
(3194, 224, 963, '1791F', 944, '1951F', 2)
(3195, 224, 1035, '1315F', 945, '1846F', 1)
(3196, 224, 945, '1846F', 970, '1687F', 0)
(3197, 224, 945, '1846F', 213, '1989F', 0)
(3198, 224, 1017, '1476E', 946, '1907F', 1)
(3199, 224, 993, '1665F', 946, '1907F', 1)
(3200, 224, 971, '1663F', 946, '1907F', 1)
(3201, 224, 984, '1691F', 946, '1907F', 1)
(3202, 224, 946, '1907F', 954, '1760F', 0)
(3203, 224, 213, '1989F', 1016, '1582F', 0)
(3204, 224, 1012, '1767F', 213, '1989F', 1)
(3205, 224, 994, '1801F', 213, '1989F', 1)
(3206, 224, 983, '1750F', 213, '1989F', 1)
(3207, 224, 947, '1912F', 1024, '1516F', 0)
(3208, 224, 999, '1623F', 947, '1912F', 1)
(3209, 224, 947, '1912F', 949, '2057F', 2)
(3210, 224, 987, '1664F', 947, '1912F', 1)
(3211, 224, 961, '1539F', 947, '1912F', 2)
(3212, 224, 947, '1912F', 952, '1843F', 2)
(3213,

(4650, 263, 1294, '2073F', 1280, '2334F', 1)
(4651, 263, 1280, '2334F', 1281, '2524F', 2)
(4652, 263, 1283, '2410F', 1280, '2334F', 2)
(4653, 263, 1280, '2334F', 194, '2336F', 0)
(4654, 263, 1281, '2524F', 1296, '2204F', 0)
(4655, 263, 1281, '2524F', 1285, '2375F', 2)
(4656, 263, 1281, '2524F', 1287, '2186F', 0)
(4657, 263, 1281, '2524F', 1284, '2294F', 2)
(4658, 263, 1288, '2244F', 1282, '2054F', 0)
(4659, 263, 1282, '2054F', 1296, '2204F', 0)
(4660, 263, 1286, '2378F', 1282, '2054F', 1)
(4661, 263, 1283, '2410F', 1294, '2073F', 2)
(4662, 263, 1283, '2410F', 1293, '2050F', 0)
(4663, 263, 1287, '2186F', 1283, '2410F', 1)
(4664, 263, 1286, '2378F', 1283, '2410F', 2)
(4665, 263, 219, '2190F', 1283, '2410F', 1)
(4666, 263, 1287, '2186F', 1284, '2294F', 2)
(4667, 263, 1284, '2294F', 194, '2336F', 0)
(4668, 263, 1284, '2294F', 1293, '2050F', 1)
(4669, 263, 1294, '2073F', 1284, '2294F', 2)
(4670, 263, 1284, '2294F', 1289, '2080F', 0)
(4671, 263, 1291, '2235F', 1284, '2294F', 1)
(4672, 263, 1

(1383, 'meunier-pion francois')
(1384, 'fruchart noe')
(1385, 'dimier serge')
(1386, 'bernard justine')
(1387, 'jacquier patrick')
(1388, 'bernard denis')
(1389, 'durteste nathan')
(1390, 'martel pierre')
(1391, 'pre flavien')
(1, 'Grand Prix de Provence 2014 - Minimes', 'mercredi 05 mars 2014 - samedi 08 mars 2014', 7, "1h30 + [30'']")
(2, 'Grand Prix de Provence 2014 - Benjamins', 'mercredi 05 mars 2014 - samedi 08 mars 2014', 7, "1h30 + [30'']")
(3, 'Grand Prix de Provence 2014 - Pupilles', 'mercredi 05 mars 2014 - samedi 08 mars 2014', 7, "1h30 + [30'']")
(4, 'Grand Prix de Provence 2014 - Open Cadets-Juniors-Qualifiés', 'mercredi 05 mars 2014 - samedi 08 mars 2014', 5, "1h30 + [30'']")
(5, 'Championnat Centre Val-de-Loire 2014 - Pupilles ', 'jeudi 06 mars 2014 - dimanche 09 mars 2014', 7, "1h30 + [30'']")
(6, 'Championnat Centre Val-de-Loire 2014 - Pupillettes', 'jeudi 06 mars 2014 - dimanche 09 mars 2014', 7, "1h30 + [30'']")
(7, 'Championnat Centre Val-de-Loire 2014 - Benjamins'

In [ ]:
#prendre un tournoi qui pose problème : les infos se décalent et régler le problème (le 30383)

In [3]:
#insertion pour berger 
#insertion de toutes les données relatives à un tournoi de type berger
def insertion_berger(number):#number=numéro du tournoi
    #insertion des joueurs du tournoi
    for player in recup_berger(number)[0]:
        recherche = list(con.execute("select name from player where name = ?", (player.lower(),)))
        if player not in recherche:
            con.execute("insert into player(name) values (?)",(player.lower(),))
    #insertion dans la table tournois        
    con.execute("insert or replace into tournois(id, name, date, rondes, cadence) values(?,?,?,?,?)",(number, infos_tournoi(number)["nom"],infos_tournoi(number)["date"],infos_tournoi(number)["nb_rondes"],infos_tournoi(number)['cadence'], ))   
    #insertion des matchs d'un tournoi
    i=0
    for player in recup_berger(number)[0]:
        dicomatch=recup_berger(number)[2]
        dicojoueur=dicomatch[player]
        joueurrestant=list(dicojoueur.keys())[i:]
        for adversaire in joueurrestant:
            black_id = list(con.execute("select id from player where name = ?", (player.lower(),)))[0][0]
            white_id = list(con.execute("select id from player where name = ?", (adversaire.lower(),)))[0][0]
            elo_black=recup_berger(number)[1][player]
            elo_white=recup_berger(number)[1][adversaire]
            if dicojoueur[adversaire]=='1':
                winner=0
            if dicojoueur[adversaire]=='0':
                winner=1
            else:
                winner=2
            con.execute("insert or replace into match(tournoi,black_player,elo_black,white_player,elo_white,winner) values(?,?,?,?,?,?)",(number,black_id,elo_black,white_id,elo_white,winner,))    
        i+=1

insertion_berger(55605)

NameError: name 'infos_tournoi' is not defined